In [1]:
# add libraries
import requests
import json
import pandas as pd
import os
import pprint

#spotipy
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Data Acquisition
### Grab data from different API sources (Spotify, Genius, Billboard 100)

In [2]:
# function for grabbing api key
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

#### Spotify

In [3]:
spotify_client = "../spotify_client"
spotify_client_secret = "../spotify_client_secret"
spotify_redirect = "../spotify_redirect"

os.environ['SPOTIPY_CLIENT_ID'] = get_file_contents(spotify_client)
os.environ['SPOTIPY_CLIENT_SECRET'] = get_file_contents(spotify_client_secret)
os.environ['SPOTIPY_REDIRECT_URI'] = get_file_contents(spotify_redirect)


spotify_artist_id = "06HL4z0CvFAxyc27GXpf02"

Endpoint usage: artist albums -> for each get album tracks -> for each track get the tracks audio features

In [4]:
# # artist albums
# scope = "user-library-read"

# sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

# results = sp.current_user_saved_tracks()
# for idx, item in enumerate(results['items']):
#     track = item['track']
#     print(idx, track['artists'][0]['name'], " – ", track['name'])

In [ ]:
scope = "user-library-read"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))

In [39]:
taylor_url = f'spotify:artist:{spotify_artist_id}'
results = sp.artist_albums(taylor_url, album_type='album', country = 'CA')

albums = results['items']

while results['next']:
    results = sp.next(results)
    albums.extend(results['items'])

# for album in albums:
#     print(album['id'])

for album in albums:
    pprint.pprint(album, width = 1)

{'album_group': 'album',
 'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02'},
              'href': 'https://api.spotify.com/v1/artists/06HL4z0CvFAxyc27GXpf02',
              'id': '06HL4z0CvFAxyc27GXpf02',
              'name': 'Taylor '
                      'Swift',
              'type': 'artist',
              'uri': 'spotify:artist:06HL4z0CvFAxyc27GXpf02'}],
 'external_urls': {'spotify': 'https://open.spotify.com/album/3lS1y25WAhcqJDATJK70Mq'},
 'href': 'https://api.spotify.com/v1/albums/3lS1y25WAhcqJDATJK70Mq',
 'id': '3lS1y25WAhcqJDATJK70Mq',
 'images': [{'height': 640,
             'url': 'https://i.scdn.co/image/ab67616d0000b273e0b60c608586d88252b8fbc0',
             'width': 640},
            {'height': 300,
             'url': 'https://i.scdn.co/image/ab67616d00001e02e0b60c608586d88252b8fbc0',
             'width': 300},
            {'height': 64,
             'url': 'https://i.scdn.co/image/ab67616d000

In [34]:
tracks = []
for album in albums:
    results = sp.album_tracks(album['id'])
    tracks.extend(results['items'])

for track in tracks:
    print(track['name'])

Lavender Haze
Maroon
Anti-Hero
Snow On The Beach (feat. Lana Del Rey)
You're On Your Own, Kid
Midnight Rain
Question...?
Vigilante Shit
Bejeweled
Labyrinth
Karma
Sweet Nothing
Mastermind
The Great War
Bigger Than The Whole Sky
Paris
High Infidelity
Glitch
Would've, Could've, Should've
Dear Reader
Lavender Haze
Maroon
Anti-Hero
Snow On The Beach (feat. Lana Del Rey)
You're On Your Own, Kid
Midnight Rain
Question...?
Vigilante Shit
Bejeweled
Labyrinth
Karma
Sweet Nothing
Mastermind
The Great War
Bigger Than The Whole Sky
Paris
High Infidelity
Glitch
Would've, Could've, Should've
Dear Reader
Lavender Haze
Maroon
Anti-Hero
Snow On The Beach (feat. Lana Del Rey)
You're On Your Own, Kid
Midnight Rain
Question...?
Vigilante Shit
Bejeweled
Labyrinth
Karma
Sweet Nothing
Mastermind
Lavender Haze
Maroon
Anti-Hero
Snow On The Beach (feat. Lana Del Rey)
You're On Your Own, Kid
Midnight Rain
Question...?
Vigilante Shit
Bejeweled
Labyrinth
Karma
Sweet Nothing
Mastermind
State Of Grace (Taylor's Versi